In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
from acnportal.acnsim import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type

In [39]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [40]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulat

In [41]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [42]:
# 25 EV sessions per weekday
events_25_ev = get_synth_events(25)

In [43]:
# Checking session details
session_copy = deepcopy(events_25_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

647 session_23 647 761 5.804009755179217 Plugin
651 session_17 651 779 8.290758981650866 Plugin
651 session_2 651 723 6.259050674687983 Plugin
658 session_8 658 719 18.989254886545574 Plugin
661 session_20 661 776 12.073856077327392 Plugin
651 session_24 651 787 5.445551659274648 Plugin
664 session_13 664 788 4.229891489469401 Plugin
676 session_15 676 761 5.374407583650504 Plugin
666 session_3 666 767 31.688361116919335 Plugin
673 session_4 673 780 12.735746816148172 Plugin
670 session_21 670 776 14.749073029037422 Plugin
657 session_6 657 846 9.056887350185892 Plugin
689 session_0 689 746 6.229656772826586 Plugin
669 session_5 669 805 14.338427519161359 Plugin
675 session_14 675 760 4.484892190650194 Plugin
722 session_1 722 770 12.689003240695111 Plugin
763 session_16 763 767 5.980409886750547 Plugin
674 session_7 674 716 5.913923400169172 Plugin
671 session_18 671 749 34.97386380336935 Plugin
815 session_19 815 841 8.547676350402641 Plugin
675 session_9 675 712 7.8495505385561035 P

#### Algorithms and new equation

In [44]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty, 1.2),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

#### Run Simulation with New Equation

In [50]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2024, 8, 19))
    end = timezone.localize(datetime(2024, 8, 19))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_25_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_1", "session_16", "session_3", "session_4", "session_17"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [51]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.33403754234313965
('RR', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 1.4804599285125732
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 14.014347314834595
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.30081844329833984
('RR', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.4678466320037842
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 14.022139310836792
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.2778434753417969
('RR', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 1.423415184020996
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 14.755212545394897
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.35613036155700684
('RR', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 1.6039369106292725
('MPC', 75)
60

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [52]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [53]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [54]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,99.209,96.0,165.000,35.554,290.340,99.000,0.407
1,three_phase,RR,60,81.194,60.0,73.333,35.562,237.618,44.000,0.422
2,three_phase,MPC,60,60.537,20.0,98.567,26.835,177.163,59.140,0.430
3,three_phase,Unctrl,65,99.209,96.0,152.308,35.554,290.340,99.000,0.423
4,three_phase,RR,65,80.946,68.0,68.954,39.866,236.893,44.820,0.440
5,three_phase,MPC,65,62.132,28.0,98.646,23.438,181.832,64.120,0.369
6,three_phase,Unctrl,70,99.209,96.0,141.429,35.554,290.340,99.000,0.394
7,three_phase,RR,70,80.924,64.0,76.493,39.837,236.827,53.545,0.416
8,three_phase,MPC,70,60.242,24.0,94.271,26.507,176.302,65.990,0.420
9,three_phase,Unctrl,75,99.209,96.0,132.000,35.554,290.340,99.000,0.420


#### Simulation with different algorithms

In [55]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty, 1.2),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [56]:
sims = dict()

In [57]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.278958797454834
('LLF', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.7016847133636475
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 16.860116481781006
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.565040111541748
('LLF', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.4324719905853271
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 21.14051127433777
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.26911497116088867
('LLF', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.7070987224578857
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 14.125792026519775
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.28693079948425293
('LLF', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.7163825035095215
('MPC', 75)
6

In [58]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [59]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,99.209,96.0,165.000,35.554,290.340,99.000,0.402
1,three_phase,LLF,60,80.918,64.0,79.900,39.836,236.809,47.940,0.362
2,three_phase,MPC,60,61.602,24.0,94.767,23.467,180.279,56.860,0.403
3,three_phase,Unctrl,65,99.209,96.0,152.308,35.554,290.340,99.000,0.403
4,three_phase,LLF,65,80.876,60.0,75.354,39.829,236.688,48.980,0.381
5,three_phase,MPC,65,62.185,28.0,99.285,23.293,181.987,64.535,0.376
6,three_phase,Unctrl,70,99.209,96.0,141.429,35.554,290.340,99.000,0.406
7,three_phase,LLF,70,80.924,64.0,74.714,39.837,236.826,52.300,0.424
8,three_phase,MPC,70,60.449,28.0,96.643,26.412,176.906,67.650,0.414
9,three_phase,Unctrl,75,99.209,96.0,132.000,35.554,290.340,99.000,0.418
